In [1]:
# https://python.langchain.com/docs/how_to/custom_embeddings/

from typing import List, Iterable, Union

from langchain_core.embeddings import Embeddings

import torch
import torch.nn.functional as F

from torch import Tensor
from transformers import AutoTokenizer, AutoModel, BitsAndBytesConfig

def _last_token_pool(last_hidden_states: Tensor,
                 attention_mask: Tensor) -> Tensor:
    left_padding = (attention_mask[:, -1].sum() == attention_mask.shape[0])
    if left_padding:
        return last_hidden_states[:, -1]
    else:
        sequence_lengths = attention_mask.sum(dim=1) - 1
        batch_size = last_hidden_states.shape[0]
        return last_hidden_states[torch.arange(batch_size, device=last_hidden_states.device), sequence_lengths]

class MyEmbeddings(Embeddings):
    def __init__(
        self,
        model: str = "Qwen/Qwen3-Embedding-8B",
        *,
        max_length: int = 8192,
        device = None
    ):
        self.model_name = model
        self.max_length = max_length

        self._tokenizer = tokenizer = AutoTokenizer.from_pretrained(self.model_name, padding_side='left')

        bnb_config = BitsAndBytesConfig(load_in_4bit=True)
        self._model = AutoModel.from_pretrained(self.model_name,quantization_config=bnb_config,).eval()

        if device is None:
            if torch.cuda.is_available():
                device = "cuda"
            elif getattr(torch.backends, "mps", None) and torch.backends.mps.is_available():
                device = "mps"
            else:
                device = "cpu"
        self._device = torch.device(device)
        self._model.to(self._device)

    def embed_documents(self, texts: List[str]) -> List[List[float]]:
        return self._embed_texts(texts)

    def embed_query(self, text: str) -> List[float]:
        return self._embed_texts([text])[0]

    @torch.inference_mode()
    def _embed_texts(self, texts: Union[List[str], Iterable[str]]) -> List[List[float]]:
        batch_dict = self._tokenizer(
            texts,
            padding=True,
            truncation=True,
            max_length=self.max_length,
            return_tensors="pt",
        )
        batch_dict.to(self._device)
        outputs = self._model(**batch_dict)
        

        embeddings = _last_token_pool(outputs.last_hidden_state, batch_dict['attention_mask'])

        out = embeddings.tolist()
        return out

In [2]:
from langchain_chroma import Chroma

embeddings = MyEmbeddings()

DB_PATH = "./data/embedded/chroma_db"

persist_db = Chroma(
    persist_directory=DB_PATH,
    embedding_function=embeddings,
    collection_name="samsung_quarterly_report",
)

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
retriever = persist_db.as_retriever()

In [4]:
from langchain_openai import ChatOpenAI

from openai import OpenAI

# Modify OpenAI's API key and API base to use vLLM's API server.
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"

messages = [
    {"role": "system", "content": "You are a helpful assistant."},
    {"role": "user", "content": "Who won the world series in 2020?"},
    {
        "role": "assistant",
        "content": "The Los Angeles Dodgers won the World Series in 2020.",
    },
    {"role": "user", "content": "Where was it played?"},
]

client = OpenAI(
    # defaults to os.environ.get("OPENAI_API_KEY")
    api_key=openai_api_key,
    base_url=openai_api_base,
)

models = client.models.list()
model = models.data[0].id

llm = ChatOpenAI(
    temperature=0.1,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    model=model
)

question = "노을이 붉은 색인 이유를 과학적으로 자세히 설명해줘"

answer = llm.invoke(question)
print(answer.content)

노을이 붉은 색을 띠는 이유는 주로 빛의 산란과 흡수 현상, 그리고 대기 중의 입자들에 의해 설명됩니다. 이를 좀 더 자세히 살펴보겠습니다.

### 1. 빛의 산란
- **레이리 산란(Rayleigh Scattering)**: 태양빛은 여러 파장의 빛으로 구성되어 있습니다. 이 빛이 지구 대기를 통과할 때, 대기 중의 분자들과 상호작용합니다.
- **파장 의존성**: 레이리 산란은 파장이 짧을수록 더 강하게 일어납니다. 즉, 파란색(약 450nm)과 보라색(약 450nm) 빛이 빨간색(약 700nm) 빛보다 더 많이 산란됩니다.
- **낮과 밤의 차이**: 태양은 하늘에서 높이 떠 있을 때, 빛이 대기를 통과하는 거리가 짧습니다. 따라서 파란색 빛이 많이 산란되어 하늘이 파랗게 보입니다.

### 2. 노을의 형성
- **태양의 위치**: 노을이 지는 시간대는 태양이 지평선 근처에 있을 때입니다. 이때 태양빛은 대기를 더 길게 통과하게 됩니다.
- **산란의 누적**: 긴 경로를 통과하면서 파란색 빛은 거의 모두 산란되고, 상대적으로 덜 산란되는 빨간색 빛이 더 많이 남아 도달하게 됩니다.
- **대기 입자의 영향**: 노을 시간대에는 대기 중의 먼지, 수증기, 그리고 기타 입자들이 더 많이 존재할 수 있습니다. 이러한 입자들은 빛을 추가적으로 산란시키고, 특히 빨간색 빛을 더 많이 통과시킵니다.

### 3. 추가적인 요인
- **대기 오염**: 대기 중의 오염 물질이나 수증기는 노을의 색을 더욱 붉게 만들 수 있습니다. 이는 빛이 더 많은 입자와 상호작용하면서 붉은 색이 강화되기 때문입니다.
- **지형적 요인**: 산이나 건물 등 지형적 요인도 노을의 색과 강도에 영향을 줄 수 있습니다.

### 결론
노을이 붉은 색을 띠는 것은 주로 태양빛이 대기를 길게 통과하면서 파란색 빛이 많이 산란되고, 상대적으로 덜 산란된 빨간색 빛이 더 많이 도달하기 때문입니다. 또한, 대기 중의 입자와 기타 환경적 요인들이 이 과정을 더욱 강화시킵니다. 이러한 과학적 원리

In [ ]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

from openai import OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
models = client.models.list()
model = models.data[0].id

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

llm = ChatOpenAI(
    model=model,
    temperature=0.1,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    )

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "이혁재 이사가 언제 선임되었지?"
response = chain.invoke(question)
print(response)

openai/gpt-oss-20b


이혁재 이사는 **2025년 3월 19일** 정기주주총회에서 선임되었습니다.


In [6]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

from openai import OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
models = client.models.list()
model = models.data[0].id
print(model)

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

llm = ChatOpenAI(
    model=model,
    temperature=0.1,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    )

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "이혁재 이사가 언제 선임되었지?"
response = chain.invoke(question)
print(response)

skt/A.X-4.0-Light
이혁재 이사가 선임된 날짜는 2025년 3월 19일입니다.


In [7]:
from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

from openai import OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
models = client.models.list()
model = models.data[0].id
print(model)

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

llm = ChatOpenAI(
    model=model,
    temperature=0.1,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    )

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "부문별 재무제표 보고서를 작성해줘"
response = chain.invoke(question)
print(response)

skt/A.X-4.0-Light
부문별 재무제표 보고서를 작성하기 위해 제공된 문서를 분석하였습니다. 아래는 삼성전자 2025년 1분기 부문별 재무제표 보고서입니다.

### 부문별 재무제표

#### 당분기 (단위: 백만원)

| 항목 | DX 부문 | DS 부문 | 기업 전체 총계 합계 |
|------|---------|---------|---------------------|
| 매출액 | 33,023,911 | 24,658,121 | 55,534,871 |
| 감가상각비 | 148,483 | 8,234,941 | 8,436,591 |
| 무형자산상각비 | 409,194 | 183,926 | 652,336 |
| 영업이익 | 1,388,471 | 82,013 | 1,469,273 |


#### 부문별 정보 설명
- **DX 부문**: 주로 소비자 전자제품 및 IT 기기를 포함하는 부문입니다.
- **DS 부문**: 반도체 및 관련 제품을 포함하는 부문입니다.
- **기업 전체 총계**: 두 부문의 합으로, 회사의 전체 재무 성과를 나타냅니다.


### 추가 정보
- **감가상각비**와 **무형자산상각비**는 각 부문별로 계산되어 있으며, 이는 자산의 사용과 관련된 비용을 나타냅니다.
- **영업이익**은 각 부문의 수익성을 평가하는 중요한 지표로, 내부거래 조정이 반영된 후의 금액입니다.
- 보고부문별 자산과 부채는 경영위원회에 정기적으로 제공되지 않아 본 보고서에 포함되지 않았습니다.


### 참고
- 이 보고서는 2025년 1분기 기준으로 작성되었습니다.
- 부문별 정보는 회사의 전략적 의사결정을 위한 경영위원회 보고용으로 사용되었습니다.


In [10]:
from dotenv import load_dotenv
load_dotenv()

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

from openai import OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
models = client.models.list()
model = models.data[0].id

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

llm = ChatOpenAI(
    model=model,
    temperature=0.1,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    )

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "부문별 재무제표 보고서를 작성해줘"
response = chain.invoke(question)
print(response)

**삼성전자 2025년 2분기 부문별 재무제표 보고서**  
(단위 : 백만원)

| 항목 | DX 부문 (기업 전체 총계) | DS 부문 (기업 전체 총계) | 기업 전체 총계 합계 |
|---|---:|---:|---:|
| **매출액** | 33,023,911 | 24,658,121 | 55,534,871 |
| **감가상각비** | 148,483 | 8,234,941 | 8,436,591 |
| **무형자산상각비** | 409,194 | 183,926 | 652,336 |
| **영업이익** | 1,388,471 | 82,013 | 1,469,273 |

> **주요 포인트**  
> 1. **DX 부문**은 2025년 2분기 매출액이 33,023,911백만원으로, 전체 매출의 약 59%를 차지합니다. 감가상각비는 148,483백만원으로 상대적으로 낮으며, 무형자산상각비는 409,194백만원입니다. 영업이익은 1,388,471백만원으로 부문별 가장 높은 수익성을 보였습니다.  
> 2. **DS 부문**은 매출액이 24,658,121백만원으로 DX 부문보다 낮지만, 감가상각비가 8,234,941백만원으로 크게 차이가 납니다. 무형자산상각비는 183,926백만원이며, 영업이익은 82,013백만원으로 DX 부문에 비해 낮은 수준입니다.  
> 3. **기업 전체**는 매출액 55,534,871백만원, 감가상각비 8,436,591백만원, 무형자산상각비 652,336백만원, 영업이익 1,469,273백만원을 기록했습니다.  
> 4. 보고서에 명시된 바와 같이, **기타 부문**은 별도로 표시되지 않았으며, 보고부문별 자산·부채는 경영위원회에 정기적으로 제공되지 않아 포함되지 않았습니다.  

> **참고**  
> - 부문별 보고는 조직 및 수익을 창출하는 제품의 유형을 기준으로 식별되며, 경영위원회는 부문에 배분될 자원과 성과를 영업이익을 기준으로 평가합니다.  
> - 본 표는 2025년 2분기(당분기) 기준이며, 전분기와의 비교는 별도 자료를 참고하시기

In [11]:
from dotenv import load_dotenv
load_dotenv()

from langchain_core.prompts import PromptTemplate
from langchain_core.output_parsers import StrOutputParser
from langchain_core.runnables import RunnablePassthrough
from langchain_openai import ChatOpenAI

from openai import OpenAI
openai_api_key = "EMPTY"
openai_api_base = "http://localhost:8000/v1"
client = OpenAI(
    api_key=openai_api_key,
    base_url=openai_api_base,
)
models = client.models.list()
model = models.data[0].id

prompt = PromptTemplate.from_template(
    """You are an assistant for question-answering tasks. 
Use the following pieces of retrieved context to answer the question. 
If you don't know the answer, just say that you don't know. 
Answer in Korean.

#Question: 
{question} 
#Context: 
{context} 

#Answer:"""
)

llm = ChatOpenAI(
    model=model,
    temperature=0.1,
    openai_api_key=openai_api_key,
    openai_api_base=openai_api_base,
    )

chain = (
    {"context": retriever, "question": RunnablePassthrough()}
    | prompt
    | llm
    | StrOutputParser()
)

question = "2025년 삼성전자의 매출실적에 대해 알려줘"
response = chain.invoke(question)
print(response)

2025년 삼성전자의 매출 실적은 2025년 1분기(제57기) 기준으로 다음과 같습니다.

| 부문 | 매출유형 | 주요 품목 | 2025년 1분기 매출(억 원) | 전년 동기 대비 증가율 |
|------|----------|-----------|---------------------------|------------------------|
| **DX 부문** | 제·상품, 용역 및 기타매출 | TV, 모니터, 냉장고, 세탁기, 에어컨, 스마트폰, 네트워크시스템, PC 등 | 517,172 | +9.4 % |
| **DS 부문** | 제·상품, 용역 및 기타매출 | DRAM, NAND Flash, 모바일 AP 등 | 251,310 | +8.6 % |
| **SDC 부문** | 제·상품, 용역 및 기타매출 | 스마트폰용 OLED 패널 등 | 58,669 | +8.9 % |
| **Harman 부문** | 제·상품, 용역 및 기타매출 | 디지털 콕핏, 카오디오, 포터블 스피커 등 | 34,194 | +6.8 % |
| **기타** | 부문간 내부거래 제거 등 |  | △69,940 | – |

- **총 매출**: 791,405억 원 (79조 1,405억원)  
- **전년 동기 대비**: 10.0 % 증가

이 수치는 2025년 5월 15일에 발표된 분기보고서(제57기)에서 확인할 수 있습니다. 2025년 전체 연간 실적은 아직 발표되지 않았으므로, 현재는 1분기 실적만을 기준으로 말씀드릴 수 있습니다.


In [10]:
from langchain_openai import ChatOpenAI
from langchain_core.prompts import ChatPromptTemplate
from langchain_core.output_parsers import StrOutputParser

prompt = ChatPromptTemplate.from_messages([
    ("system", "You are a helpful assistant. Please respond to the user's request only based on the given context."),
    ("user", "Question: {question}\nContext: {context}")
])
model = ChatOpenAI(model="gpt-4o-mini")
output_parser = StrOutputParser()

chain = prompt | model | output_parser

question = "Can you summarize this morning's meetings?"
context = "During this morning's meeting, we solved all world conflict."
chain.invoke({"question": question, "context": context})

"This morning's meeting was highly productive, as we successfully addressed and resolved all world conflicts."